In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [111]:
data = pd.read_csv('Hackathon Raw Files/Full Datasets - Opta/MLS/MLS 2017-2018.csv')

/Users/benedictbrady/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (288,298) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Pass Features

In [112]:
data['pass_x'] = data['pass_end_x'] - data['x']
data['pass_y'] = data['pass_end_y'] - data['y']

In [113]:
#df_pass = data.query('event_type == "Pass"')

#fig, ax = plt.subplots(1, 1, figsize = (10, 10))
#ax.scatter(df_pass['pass_x'], df_pass['pass_y'])

df_new = data.pivot_table(index = ['player', 'player_id'], values=['pass_x', 'pass_y'], aggfunc=np.mean).reset_index()
df_new = df_new.set_index('player')

#### Offsides Player Counts 

In [114]:
list(data.columns)
#data.query('players_caught_offside')['event_type']#.unique()
df_off = data[data['players_caught_offside'].isnull() == False].pivot_table(index = \
                                            'players_caught_offside', values = 'player', aggfunc = len)
df_new = df_new.merge(df_off, left_on='player_id', right_index=True, how = 'left')

#### Foul Count

In [115]:
#data.query('event_type == "Foul"')
df_foul = data.query('event_type == "Foul"').pivot_table(index = 'player', values = 'x', aggfunc = len)
df_new = df_new.merge(df_foul, left_index=True, right_index=True, how = 'left')

In [116]:
df_new.columns = ['player_id', 'pass_x', 'pass_y', 'offsides_count', 'fouls']

In [117]:
df_new = df_new.drop(['pass_x', 'pass_y'], axis = 1)

#### Aerial

In [118]:
df_head_def = data.query('event_type == "Aerial" & x < 30').pivot_table(index = 'player', \
                                        values = 'outcome', aggfunc = [np.mean, len])
df_head_mid = data.query('event_type == "Aerial" & x >= 30 & x < 70').pivot_table(index = 'player', \
                                        values = 'outcome', aggfunc = [np.mean, len])
df_head_off = data.query('event_type == "Aerial" & x >= 70').pivot_table(index = 'player', \
                                        values = 'outcome', aggfunc = [np.mean, len])

df_head = df_head_def.merge(df_head_mid, right_index = True, left_index = True, how = 'outer')
df_head = df_head.merge(df_head_off, right_index = True, left_index = True, how = 'outer')
df_head.columns = ['Header Defense Mean', 'Header Defense Count', 'Header Midfield Mean', \
                   'Header Midfield Count', 'Header Offense Mean', 'Header Offense Count']

df_new = df_new.merge(df_head, left_index=True, right_index=True, how = 'left')

#### Interceptions

In [119]:
df_int = data.query('event_type == "Interception"').pivot_table(index = 'player', values = 'x', aggfunc = len)
df_int.columns = ['Interceptions']
df_new = df_new.merge(df_int, left_index=True, right_index=True, how = 'left')

#### Long Passes 

In [120]:
df_lb = data.query('event_type == "Pass" and long_ball == 1').pivot_table(index = 'player', values = 'outcome', aggfunc = [np.mean, len])
df_lb.columns = ['Long Balls Mean', 'Long Balls Count']
df_new = df_new.merge(df_lb, left_index=True, right_index=True, how = 'left')

#### Take Ons 

In [121]:
data['abs y'] = abs(50 - data['y'])

takeondf=data.query('event_type == "Take on"').pivot_table(index='player', values=['x', 'y', 'abs y'], aggfunc=[np.mean,len])

numsuccess=data.query('event_type == "Take on" and outcome==1').pivot_table(index='player', values='x', aggfunc=[len])

numfailure=data.query('event_type == "Take on" and outcome!=1').pivot_table(index='player',values='x', aggfunc=[len])

takeondf = takeondf.merge(numsuccess, left_index=True, right_index=True, how = 'outer')
takeondf = takeondf.merge(numfailure, left_index=True, right_index=True, how = 'outer')

takeondf.columns = ['Take On Mean X', 'Take On Mean Y', 'Take On Mean Y Abs', 'Take On Total', 'Extra 1', 'Extra 2', 'Take On Success', 'Take On Failure']
takeondf = takeondf.drop(['Extra 1', 'Extra 2'], axis = 1)

In [122]:
df_new = df_new.merge(takeondf, left_index=True, right_index=True, how = 'left')

In [123]:
df_new.to_csv('featuresBenedict3.csv')

In [125]:
df_new.columns

Index(['player_id', 'offsides_count', 'fouls', 'Header Defense Mean',
       'Header Defense Count', 'Header Midfield Mean', 'Header Midfield Count',
       'Header Offense Mean', 'Header Offense Count', 'Interceptions',
       'Long Balls Mean', 'Long Balls Count', 'Take On Mean X',
       'Take On Mean Y', 'Take On Mean Y Abs', 'Take On Total',
       'Take On Success', 'Take On Failure'],
      dtype='object')